# Image Restoration
It is a process to recover an image that has been degraded by using a priori knowledge of the degradation phenomenon.

## Basic Concepts

### General Workflow
<p align="center"> 
<img src="img/workflow.png" />
</p>
### The degraded image $g(x, y)$ is given by

#### Spatial domain:  
$$g(x, y) = h(x, y) \star f(x, y) + \eta(x, y)$$

#### Frequency domain: 
$$G(u, v) = H(u, v)F(u, v) + N(u, v)$$

In [2]:
import cv2

## Noise Models
Noise is a random variable, therefore, it can be represented by probability density function (p.d.f)

### Assumptions
* Noise is independent of spatial coordinates
* Noise is uncorrelated with respect to the image (pixel values) 

### Sources of the noise
* Image acquisition
    * signal fluctuations in the detector
     
* Image transmission
    * data corruption
  
### Common noise PDF
#### Gaussian
* Electronic circuit noise, sensor noise
$$
    p(z) = \frac{1}{\sqrt{2\pi}\sigma} e^{-(z-\bar{z})^2/2\sigma^2}
    \\ \mu = \bar{z}, \sigma^2 = \sigma^2
$$
    
#### Rayleigh
* Range imaging
$$
    p(z) = 
    \left\{
    \begin{array}{ll}
            \frac{2}{b} (z - a) e^{-(z-a)^2/b}& \text{for z}  \geq a \\
            0 & \text{for z  < a}
    \end{array} 
    \right.
    \\ \mu = \sigma^2 = a + \sqrt{\pi b/4}
$$

#### Erlang (Gamma)
* Laser imaging
$$
    p(z) = 
    \left\{
    \begin{array}{ll}
            \frac{a^bz^{b-1}}{(b - 1)!} e^{-az} & \text{for z}  \geq 0 \\
            0 & \text{for z  < 0}
    \end{array} 
    \right.
    \\ \mu = \frac{b}{a}, \sigma^2 = \frac{b}{a^2}
$$

#### Expoential
* Laser imaging
$$
    p(z) = 
    \left\{
    \begin{array}{ll}
            ae^{-az} & \text{for z}  \geq 0 \\
            0 & \text{for z  < 0}
    \end{array} 
    \right.
    \\ \mu = \frac{1}{a}, \sigma^2 = \frac{1}{a^2}
$$
    
#### Uniform
$$
    p(z) = 
    \left\{
    \begin{array}{ll}
            \frac{1}{b - a}& \text{for z}  \geq 0 \\
            0 & \text{for z  < 0}
    \end{array} 
    \right.
    \\ \mu = \frac{a + b}{2}, \sigma^2 = \frac{(b - a)^2}{12}
$$
    
#### Impulse (salt and pepper noise)
* Quick transients
$$
    p(z) = 
    \left\{
    \begin{array}{ll}
        P_a& \text{for z = a (pepper)} \\
        P_b & \text{for z = b (salt)} \\
        0 & \text{otherwise}
    \end{array} 
    \right.
    \\ \text{where b > a}
$$

## Noise removal in spatial domain
Suppose we know the degradation function, then we just need to subtract the noise from $g(x, y)$

### Mean filters

#### Arithmetic Mean Filter
* good for random noise (gaussian, uniform...)
* Smooth the local variation, result in blurring in image

$$
     \hat{f}(x, y) = \frac{1}{mn} \sum_{(s, t) \in S_{xy}} g(s, t)    
$$

#### Geometric Mean Filter
* better performance in random noise than arithmetic mean filter
* fails for pepper noise

$$
     \hat{f}(x, y) = 
         \left(  
                 \prod_{(s, t) \in S_{xy}} g(s, t)  
         \right)^{\frac{1}{mn}}
$$

#### Harmonic Mean Filter
* good for gaussian and salt noise, but fails for pepper noise

$$
    \hat{f}(x, y) = 
         \frac{mn}{\sum_{(s, t) \in S_{xy}} \frac{1}{g(s, t)}}
$$

#### Contraharmonic Mean Filter
* It can eliminate salt and pepper noise, but not both
* positive Q for pepper noise
* negative Q for salt noise

$$
     \hat{f}(x, y) = 
         \frac{\sum_{(s, t) \in S_{xy}} g(s, t)^{Q+1}}{\sum_{(s, t) \in S_{xy}} g(s, t)^{Q}}
     \\ \text{when Q = 0, it is arithmetic mean, when Q = -1, it is harmonic mean}    
$$

### Order-statistics filters
Basic idea: Arranges all pixels in sequential order 

#### Min and max
* min filter good for salt

$$ \hat{f}(x, y) = \min_{(s, t) \in S_{xy}}{g(s, t)}$$

* max filter good for pepper

$$ \hat{f}(x, y) = \max_{(s, t) \in S_{xy}}{g(s, t)}$$

#### Median filter
* Super good for impulse noise
* Better than average filter as not 

$$ \hat{f}(x, y) = median_{(s, t) \in S_{xy}}{g(s, t)}$$

#### Midpoint filter
* Useful for Gaussian and uniform noise
* not good for impulse signal (impulse signal is usually max/min value in the images, think about taking average between max/min)

$$ \hat{f}(x, y) = \frac{1}{2} (\min_{(s, t) \in S_{xy}}{g(s, t)} + \max_{(s, t) \in S_{xy}}{g(s, t)})$$

#### Alpha-trimmed mean filter
* Useful for combination noise such as salt-and-pepper with Gaussian noise

$$ 
    \hat{f}(x, y) = \frac{1}{mn - d} \sum_{(s, t) \in S_{xy}}{g_r(s, t)} \\
   \text{where largest/smallest} \frac{d}{2} \text{ element have to be deleted in} S_{xy} 
$$

### Adaptive mean filters
Aims to preserve edges 
* if $\sigma^2_{\eta}$ is zero => no noise, return $g(x, y)$ because $g(x, y) = f(x, y)$ 
* if $\sigma^2_{L}$ is high relative to $sigma^2_{\eta}$ => edges, return value closed to $g(x, y)$
* if $\sigma^2_{L} = \sigma^2_{\eta}$  => areas inside object, return the arithmetic mean

$$
    \hat{f}(x, y) = g(x, y) - \frac{\sigma^2_{\eta}}{\sigma^2_{L}} [g(x, y) - m_{L}] \\
    
    \text{where } \sigma^2_{\eta} \text{ is global noise variance, } \sigma^2_{L} \text{ is local variance, } m_{L} \text{ is the local average}
$$
